# 1. Environment Setup

In [1]:
import math
import numpy as np
import pandas as pd

# 2. Data Acquisition

### 2.1 Variables Definition

In [2]:
alloy = ['all_1','all_2','all_3','all_4','all_5']
supplier = ['sup_1','sup_2','sup_3','sup_4','sup_5']
product = ['prod_1','prod_2','prod_3']
month = ['Init', 'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
I = len(alloy)
J = len(supplier)
Z = len(product)
M = len(month)
q = 0.05
mu = 1

### 2.2 Data Filepath

In [3]:
file_loc = 'Data Templates.xlsx'

### 2.3 Contract Upper Threshold

In [4]:
contract_upper_threshold = pd.read_excel(file_loc,sheet_name='Contract Upper Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_upper_threshold.index=alloy

FileNotFoundError: [Errno 2] No such file or directory: 'Data Templates.xlsx'

In [ ]:
h = contract_upper_threshold
contract_upper_threshold

### 2.4 Contract Lower Threshold

In [ ]:
contract_lower_threshold = pd.read_excel(file_loc,sheet_name='Contract Lower Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_lower_threshold.index=alloy

In [ ]:
l = contract_lower_threshold
contract_lower_threshold

### 2.5 Contract Penalties

In [ ]:
contract_penalties = pd.read_excel(file_loc,sheet_name='Contract Penalties',header=2, na_values=['NA'], usecols="B:F")
contract_penalties.index=['Penalty Fee %']

In [ ]:
p = contract_penalties
contract_penalties

### 2.6 Annual Contract Limit

In [ ]:
annual_contracted_limit= pd.read_excel(file_loc,sheet_name='Annual Contracted Limit',header=2, na_values=['NA'], usecols="B:F")
annual_contracted_limit = annual_contracted_limit.iloc[:5,:]
annual_contracted_limit.index=alloy

In [ ]:
k = annual_contracted_limit
annual_contracted_limit

### 2.7 Unit Cost

In [ ]:
unit_cost = pd.read_excel(file_loc,sheet_name='Unit Cost',header=2, na_values=['NA'], usecols="B:F")
unit_cost = unit_cost.iloc[:5,:]
unit_cost.index=alloy

In [ ]:
c = unit_cost
unit_cost

### 2.8 Product Revenue

In [ ]:
prod_rev = pd.read_excel(file_loc,sheet_name='Prod Rev',header=2, na_values=['NA'], usecols="B:F")
prod_rev.index=['Revenue ($)']

In [ ]:
r = prod_rev
prod_rev

### 2.9 Product Recipe

In [ ]:
prod_recipe= pd.read_excel(file_loc,sheet_name='Prod Recipe',header=2, na_values=['NA'], usecols="B:D")
prod_recipe = prod_recipe.iloc[:5,:]
prod_recipe.index=alloy

In [ ]:
u = prod_recipe
prod_recipe

### 2.10 Predicted Demand

In [ ]:
predicted_demand= pd.read_excel(file_loc,sheet_name='Predicted Demand',header=2, na_values=['NA'], usecols="B:D")
predicted_demand.index=month

In [ ]:
d = predicted_demand
predicted_demand

# 3. Baseline Model

In [ ]:
alloy_demand = predicted_demand @ prod_recipe.T
alloy_demand

In [ ]:
cap_tracker = annual_contracted_limit.copy()

def order(a):
    orders = pd.DataFrame(np.zeros(M, S), index=month, columns=supplier)
    cap_tracker = annual_contracted_limit.loc[a].copy()
    
    for m in month:
        dem = alloy_demand.loc[m, a]
        sup_prices = unit_cost.loc[a]
        thresh_tracker = contract_upper_threshold.loc[a].copy()
        
        while dem > 0:            
            for s in supplier:
                if cap_tracker[s] == 0:
                    # Prohibit buying from suppliers with depleted stock for the year
                    sup_prices[s] = np.inf

                elif thresh_tracker[s] <= 0:
                    # Update pricing if month threshoold exceeded
                    sup_prices[s] = unit_cost.loc[a, s]*(1+contract_penalties[s])

            s_min = sup_prices.idxmin()

            if thresh_tracker[s_min] > 0: # have not exceeded monthly threshold yet
                # make an order that does not exceed monthly threshold, remaining demand, and remaining annual capacity
                order = min(thresh_tracker[s_min], cap_tracker[s_min], dem)
                # Update trackers
                dem -= order
                cap_tracker[s_min]-=order
                thresh_tracker[s_min]-=order
                orders.loc[m, s_min]+=order

            else: # Monthly theshold exceeded but still best option
                order = min(cap_tracker[s_min], dem)
                # Update trackers
                dem -= order
                cap_tracker[s_min]-=order
                thresh_tracker[s_min]-=order
                orders.loc[m, s_min]+=order
            
    return orders